# CouchDB CRUD operace prostřednictvím GraphQL

## Problémy v projektu

  ### Nefunkční soubor run.bat

Při spuštění souboru run.bat nastala chyba v prostředí Dockeru, konkrétně v oblasti import module 'couchdb', ale při lokálním spuštění proběhlo vše 
bez problémů. Domnívali jsme se, že je zřejmě chyba v requirements.

<b>Řešení:</b>

V requirements jsme se snažili nainstalovat dvě knihovny a to knihovnu CouchBase a CouchDB, které spolu však nespolupracují. Odstraněním knihovny CouchBase byl problém vyřešen.


### Chyba načítaní schémat v GQL

Při načítání schémat v GQL nastala chyba.


<b>Řešení:</b>

Vyřešeno vytvořením nových schémat, modelů, queries a resloverů.


### UpdateUser chyba

<b>Řešení:</b>